# Transformations of 2D image

Assuming the image is represent by 2D array:

Coordinate matrix:

```
size: 4 x 5
(0, 0) (1, 0) (2, 0) (3, 0)
(0, 1) (1, 1) (2, 1) (3, 1)
(0, 2) (1, 2) (2, 2) (3, 2)
(0, 3) (1, 3) (2, 3) (3, 3)
(0, 4) (1, 4) (2, 4) (3, 4)
```

## Turn 90 degrees

### Right

We want to achieve the following transformation of coordinates:

```
size: 5 x 4
(0, 4) (0, 3) (0, 2) (0, 1) (0, 0)
(1, 4) (1, 3) (1, 2) (1, 1) (1, 0)
(2, 4) (2, 3) (2, 2) (2, 1) (2, 0)
(3, 4) (3, 3) (3, 2) (3, 1) (3, 0)
```

So as we can see, it comes to:

$$ (x_r, y_r) \to (size_y - y - 1, x) $$

### Left

We want to achieve the following transformation of coordinates:

```
size: 5 x 4
(3, 0) (3, 1) (3, 2) (3, 3) (3, 4)
(2, 0) (2, 1) (2, 2) (2, 3) (2, 4)
(1, 0) (1, 1) (1, 2) (1, 3) (1, 4)
(0, 0) (0, 1) (0, 2) (0, 3) (0, 4)
```

It comes to:

$$ (x_l, y_l) \to (y, size_x - x - 1) $$

In [1]:
def right_90_transform(x: int, y: int, size_x: int, size_y: int) -> tuple:
    return size_y - y - 1, x

def left_90_transform(x: int, y: int, size_x: int, size_y: int) -> tuple:
    return y, size_x - x - 1

## Flips

### Vertical

```
size: 4 x 5
(0, 4) (1, 4) (2, 4) (3, 4)
(0, 3) (1, 3) (2, 3) (3, 3)
(0, 2) (1, 2) (2, 2) (3, 2)
(0, 1) (1, 1) (2, 1) (3, 1)
(0, 0) (1, 0) (2, 0) (3, 0)
```

It comes to:

$$ (x_h, y_h) \to (x, size_y - y - 1) $$

### Horizontal

```
size: 4 x 5
(3, 0) (2, 0) (1, 0) (0, 0)
(3, 1) (2, 1) (1, 1) (0, 1)
(3, 2) (2, 2) (1, 2) (0, 2)
(3, 3) (2, 3) (1, 3) (0, 3)
(3, 4) (2, 4) (1, 4) (0, 4)
```

It comes to:

$$ (x_v, y_v) \to (size_x - x - 1, y) $$

In [2]:
def flip_vertical(x: int, y: int, size_x: int, size_y: int) -> list:
    return x, size_y - y - 1

def flip_horizontal(x: int, y: int, size_x: int, size_y: int) -> list:
    return size_x - x - 1, y


## Representation

Possible representation that kind of table:
* as 1D table
* as table of tables
* as **dictionary** with tuple as keys
* as **set** of tuples (representing coordinates) if picture is "bitmap" (contain only two colours)

In Python the two last options are worth considering.

In [3]:
def load_pixels(image: list) -> tuple:
    for row, line in enumerate(image):
        yield from ((column, row, c) for column, c in enumerate(line))

### Image represent as 1D array

If know the size of picture, can calculate the coordinates of projection:

$$ p_x \to y * size_y + x $$

The reverse projection:

$$
\begin{split}
    x & \to p_x \pmod{size_x} \\
    y & \to \left \lfloor{\frac{p_x}{size_x}}\right \rfloor
\end{split}
$$

In Python the most fitting library for it is [Array](https://docs.python.org/3/library/array.html#module-array).

In [4]:
import array

def to_array_image(image: list) -> array:
    pixels = list(load_pixels(image))
    size_column = max(column for column, _, _ in pixels) + 1
    size_row = max(row for _, row, _ in pixels) + 1

    return array.array('u', [value for _, _, value in pixels]), size_column, size_row

def print_array_image(image: tuple):
    pixels, size_column, size_row = image

    for row in range(0, size_row):
        current_row_index = row * size_column
        print(''.join(pixels[current_row_index:current_row_index + size_column]))

def transform_array_image(transform, image, new_size_x, new_size_y) -> set:
    pixels, old_size_x, old_size_y = image

    results = array.array('u', pixels)
    for i, c in enumerate(pixels):
        x_o, y_o = i % old_size_x, i // old_size_x
        x_n, y_n = transform(x_o, y_o, old_size_x, old_size_y)
        results[y_n * new_size_x + x_n] = c

    return results, new_size_x, new_size_y

alphabet = to_array_image('''abcd
efgh
ijkl'''.splitlines())

print('Original picture:\n')
print_array_image(alphabet)

print('\nRotate 90 right of original:\n')
print_array_image(transform_array_image(right_90_transform, alphabet, 3, 4))

print('\nRotate 90 left of original:\n')
print_array_image(transform_array_image(left_90_transform, alphabet, 3, 4))

print('\nFlip vertical of original:\n')
print_array_image(transform_array_image(flip_vertical, alphabet, 4, 3))

print('\nFlip horizontal of original:\n')
print_array_image(transform_array_image(flip_horizontal, alphabet, 4, 3))

Original picture:

abcd
efgh
ijkl

Rotate 90 right of original:

iea
jfb
kgc
lhd

Rotate 90 left of original:

dhl
cgk
bfj
aei

Flip vertical of original:

ijkl
efgh
abcd

Flip horizontal of original:

dcba
hgfe
lkji


### Image represent as table of tables

The easiest way of presenting matrix (2D array) in Python is list of lists. 

In [5]:
def to_matrix_image(image: list) -> list:
    pixels = list(load_pixels(image))
    size_column = max(column for column, _, _ in pixels) + 1
    size_row = max(row for _, row, _ in pixels) + 1

    result = [[0] * size_column for i in range(size_row)]
    for x, y, c in pixels:
        result[y][x] = c

    return result, size_column, size_row

def print_matrix_image(image: tuple):
    pixels, size_column, size_row = image

    for row in range(0, size_row):
        print(''.join(pixels[row]))

def transform_matrix_image(transform, image, new_size_x, new_size_y) -> set:
    pixels, old_size_x, old_size_y = image

    result = [[0] * new_size_x for i in range(new_size_y)]
    for y_o in range(old_size_y):
        for x_o in range(old_size_x):
            x_n, y_n = transform(x_o, y_o, old_size_x, old_size_y)
            result[y_n][x_n] = pixels[y_o][x_o]

    return result, new_size_x, new_size_y

alphabet = to_matrix_image('''abcd
efgh
ijkl'''.splitlines())

print('Original picture:\n')
print_matrix_image(alphabet)

print('\nRotate 90 right of original:\n')
print_matrix_image(transform_matrix_image(right_90_transform, alphabet, 3, 4))

print('\nRotate 90 left of original:\n')
print_matrix_image(transform_matrix_image(left_90_transform, alphabet, 3, 4))

print('\nFlip vertical of original:\n')
print_matrix_image(transform_matrix_image(flip_vertical, alphabet, 4, 3))

print('\nFlip horizontal of original:\n')
print_matrix_image(transform_matrix_image(flip_horizontal, alphabet, 4, 3))

Original picture:

abcd
efgh
ijkl

Rotate 90 right of original:

iea
jfb
kgc
lhd

Rotate 90 left of original:

dhl
cgk
bfj
aei

Flip vertical of original:

ijkl
efgh
abcd

Flip horizontal of original:

dcba
hgfe
lkji


### Image represent as points in set

This representation is usefull for _black and white_ pictures (here I will use the `_` and `#`). The **set** contains just the bits of one colour. If set don't contain the certain point, it means that is the other colour.

The advantage is that set are very fast for checks if point below or not to the picture.

In [6]:
def to_set_image(image: list) -> set:
    return set((column, row) for column, row, c in load_pixels(image) if c == '#')

def print_set_image(image: set):
    max_column = max(column for column, _ in image)
    max_row = max(row for _, row in image)

    for row in range(0, max_row + 1):
        for column in range(0, max_column + 1):
            print('#' if (column, row) in image else ' ', end='')
        print()

arrow = to_set_image('''# # 
  ##
####'''.splitlines())

def transform_set_image(transform, picture: set, size_x: int, size_y: int) -> set:
    return set(transform(x, y, size_x, size_y) for x, y in picture)

print('Original picture:\n')
print_set_image(arrow)

print('\nRotate 90 right of original:\n')
print_set_image(transform_set_image(right_90_transform, arrow, 4, 3))

print('\nRotate 90 left of original:\n')
print_set_image(transform_set_image(left_90_transform, arrow, 4, 3))

print('\nFlip vertical of original:\n')
print_set_image(transform_set_image(flip_vertical, arrow, 4, 3))

print('\nFlip horizontal of original:\n')
print_set_image(transform_set_image(flip_horizontal, arrow, 4, 3))

Original picture:

# # 
  ##
####

Rotate 90 right of original:

# #
#  
###
## 

Rotate 90 left of original:

 ##
###
  #
# #

Flip vertical of original:

####
  ##
# # 

Flip horizontal of original:

 # #
##  
####


### Image represent as dictionary

The representation is keeping points as key, and the pixel as value for the key.


In [7]:
def to_dict_image(image: list) -> dict:
    return {(column, row): c for column, row, c in load_pixels(image)}

def print_dict_image(image: dict):
    max_column = max(column for column, _ in image.keys())
    max_row = max(row for _, row in image.keys())

    for row in range(0, max_row + 1):
        for column in range(0, max_column + 1):
            print(image[column, row], end='')
        print()

def transform_dict_image(transform, picture: set, size_x: int, size_y: int)  -> set:
    return {transform(x, y, size_x, size_y):picture[x, y] for x, y in picture.keys()}

alphabet = to_dict_image('''abcd
efgh
hijk'''.splitlines())

print('Original picture:\n')
print_dict_image(alphabet)
print()

print('\nRotate 90 right:\n')
print_dict_image(transform_dict_image(right_90_transform, alphabet, 4, 3))

print('\nRotate 90 left:\n')
print_dict_image(transform_dict_image(left_90_transform, alphabet, 4, 3))

print('\nFlip vertical of original:\n')
print_dict_image(transform_dict_image(flip_vertical, alphabet, 4, 3))

print('\nFlip horizontal of original:\n')
print_dict_image(transform_dict_image(flip_horizontal, alphabet, 4, 3))

Original picture:

abcd
efgh
hijk


Rotate 90 right:

hea
ifb
jgc
khd

Rotate 90 left:

dhk
cgj
bfi
aeh

Flip vertical of original:

hijk
efgh
abcd

Flip horizontal of original:

dcba
hgfe
kjih


## All transformations combination

The question of this chapter is: if picture can be rotated and fliped, how many unique pictures are produced.
